<a href="https://colab.research.google.com/github/nanopiero/exam_S3/blob/master/notebooks/probleme_et_consignes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Examen de Machine Learning - Session 3


In [ ]:
# Imports des bibliothèques utiles
# pour l'IA
import torch
# pour les maths
import numpy as np
# pour afficher des images et des courbes
import matplotlib.pyplot as plt

In [ ]:
! git clone https://github.com/nanopiero/exam_S3.git

In [ ]:
pip install einops

## I. Découverte du problème

Dans ce problème, il va s'agir de reconstruire un champ 2D à partir de plusieurs sources d'information. Les sources d'information sont les suivantes :
  - des mesures ponctuelles du champ 2D
  - un prédicteur spatialisé, qui consiste en un champ 2D bruité.
  - des mesures par tomographie obtenues le long de segments

Le but est de coder et d'entamer la comparaison entre deux méthodes d'apprentissage différentes basées sur des réseaux de neurones profonds. Par souci de simplicité, nous allons travailler sur des données de synthèse générées à la volée.

Ces données peuvent être visualisées grâce à la fonction *gen_image_with_pairs*.

In [ ]:
from exam_S3.utile_Transformers import voir_batch2D, gen_image_with_pairs, set_tensor_values

batch_size = 6
n_points = 16
n_pairs = 16
full_target, point_measurements, spatial_predictor, line_measurements_viz, line_measurements = gen_image_with_pairs(6, n_pairs, n_points)
# NB : - Le code de gen_image_with_pairs est précompilé avec numba. Le premier run est donc nettement plus long que les suivants.
#      - Le terme "pairs" dans *gen_images_with_pairs* fait référence aux extrémités des segments.

In [ ]:
# exemples de champ 2D cible complets (full_target)
# ils contiennent des disques, qu'il va s'agir de reconstruire au mieux
fig1 = plt.figure(1, figsize=(36, 6))
voir_batch2D(full_target, 6, fig1, k=0, min_scale=0, max_scale=1)

In [ ]:
# Pour reconstruire full_target, on s'appuira sur des triplets contenant les positions et les
# valeurs de mesures ponctuelles (point_measurements).
# Précisément, ces triplets (x, y, m) contiennent :
# - les coordonnées x, y des mesures ponctuelles dans le repère (O, A, B)
# où O correspond au coin en bas à gauche de full_target, A au coin en bas à droite
# et B au coin en haut à gauche.
# - m : valeur au pixel de coordonnées (x,y) de full_target

# Nous avons généré batch_size x n_points triplets :
print(point_measurements.shape)

# Pour visualiser ces mesures, on peut utiliser la fonction set_tensor_values(t,point_measurements, size):
# qui affectent aux pixels de t de coordonnées x,y les valeurs m. Par exemple:
point_measurements_viz = set_tensor_values( - 0.1 * torch.ones((6,1,64,64)), point_measurements, 64)
fig2 = plt.figure(2, figsize=(36, 6))
voir_batch2D(point_measurements_viz , 6, fig2, k=0, min_scale= -0.1, max_scale=0.5)
# NB: bien noter le format utilisé pour le tenseur t

In [ ]:
# On s'appuiera aussi sur des prédicteurs spatialisés bruités.
# Les rectangles figurent le bruit. Les disques contenus dans ces images
# sont alignés avec ceux du champ 2D à reconstruire
# mais leurs intensités sont différentes.

fig3 = plt.figure(3, figsize=(36, 6))
voir_batch2D(spatial_predictor, 6, fig3, k=0, min_scale=0, max_scale=1)

In [ ]:
# Enfin, on s'appuiera sur des mesures intégrées le long de segments.
# Les positions de ces segments et les mesures associées sont contenues
# dans la variable line_measurements_viz.
# Précisément, line_measurements_viz contient des quintuplets (x0, y0, x1, y1, Is) où :
# - les coordonnées x0, y0 de la première extrémité du segment
# - les coordonnées x1, y1 de la seconde extrémité du segement
# - la valeur moyenne Is du champ 2D full_target le long du segment.


# Au-dessus, nous avons généré 6 x 16 quintuplets :
print(line_measurements.shape)


# Le tenseur line_measurements_viz permet de visualiser ces segments :
fig3 = plt.figure(3, figsize=(36, 6))
voir_batch2D(line_measurements_viz, 6, fig3, k=0, min_scale=0, max_scale=1)

# NB: dans line_measurements_viz, les intensités des pixels par lesquels passent
# les segments ont été réglées sur 0.2 + Is  (sauf aux intersections)

## II. Consignes

La fonction *gen_image_with_pairs* permet d'aborder plusieurs problèmes d'apprentissage par plusieurs méthodes différentes.
Notons $Y_c$, la cible complète spatialisée, $X_p$ l'ensemble des mesures ponctuelles disponibles, $X_s$ le prédicteur spatialisé et bruité,  $X_i$ l'ensemble des mesures intégrées selon des segments et $X^{2D}_i$ leur représentation sous forme de champ 2D (c'est à dire la variable *line_measurements_viz*).
Les problèmes de **régression** auxquels nous allons nous intéresser sont :

$\mathcal{P}_{0}$ : $X_{s}  \rightarrow  Y_c$ \
$\mathcal{P}_{1}$ : $(X_{s}, X^{2D}_i)  \rightarrow  Y_c$

$\mathcal{P}_{2}$ : $(X_{s}, X_i)  \rightarrow  Y_c$ \
$\mathcal{P}_{3}$ : $(X_{s}, X_p, X_i)  \rightarrow  Y_c$ \
$\mathcal{P}_{4}$ : $X_i  \rightarrow  X^{2D}_i$


Le travail à faire se partage en deux parties:
  - Aborder $\mathcal{P}_{0}$ et $\mathcal{P}_{1}$ avec un U-Net. Montrer que le réseau peut prendre en compte les mesures intégrées le long des segments pour améliorer les performances. \
  La preuve de concept peut être faite avec un U-Net relativement léger (voir Annexe A. ci-dessous). Pour $\mathcal{P}_{1}$, les entrées seront les images à deux canaux formées par concaténation de $X_{s}$ et $X^{2D}_i$.
  Utiliser une fonction de coût simple (eg MAE), un optimizer standard.

  - Aborder $\mathcal{P}_{2}$, $\mathcal{P}_{3}$ et $\mathcal{P}_{4}$ avec des *visual transformers* adaptés à des données d'entrées multimodales.
  L'annexe B, ci-dessous fournit un exemple basique d'un tel transformer adapté au problème de régression $\mathcal{P}_{3}$. C'est un ViT qu'on a légèrement modifié :
    - pour de la prédiction dense, la cible $Y_c$ étant spatialisée (voir les convolution transposées du module *utile_Transformers*).
    - pour prendre en compte les mesures ponctuelles et les mesures intégrées sans les encoder dans une image (voir la classe *UnifiedEmbedding*).
    
  Dans cette seconde partie, il s'agit alors :
    - d'entraîner sur quelques centaines d'époques un tel visual transformer sur
  $\mathcal{P}_{2}$ et $\mathcal{P}_{3}$.
    - de comparer les résultats avec ceux de la partie précédente aux plans quantitatif et qualitatif.
    - d'aborder $\mathcal{P}_{4}$ pour évaluer la capacité d'un tel Visual Transformer à passer d'un encodage des mesures intégrées sous forme de quintuplets à leur représentation 2D.

Le rendu consistera en un répertoire Github public avec deux à trois notebooks contenant des résultats reproductibles et des modules python annexes avec le code nécessaire. \
Par reproductible, j'entends que les poids après entraînement doivent être mis à  disposition, par exemple sur un site de transfert de dossier comme GrosFichier.com. Ils doivent pouvoir être chargés dans les notebooks (voir Annexe C).


Autres éléments de consigne:

- Toute prise d'initiative visant à faciliter l'apprentissage (optimisation des hyperparamètres, utilisation d'un *scheduler*, utilisation d'un Visual Transformer mieux adapté à une régression par pixel, etc) sera valorisée.
- Expliquer votre démarche et n'hésitez pas à aborder d'autres problèmes de régression que ceux évoqués ci-dessus, si vous pensez que cela peut avoir un intérêt.

Conseils:
  - Eviter de trop pousser vos apprentissages : les ressources sous colab sont limitées. Quelques centaines d'époques (avec une époque = 64 images) suffiront. Pour l'apprentissage des UNet, 100 époques suffisent largement.
  - Ne pas hésiter à poser des questions par mail: l'énoncé est assez ouvert et pourra être ajusté en fonction des points de blocage éventuels. Contact: pierre.lepetit@meteo.fr.
  - Il est fortement conseillé de (re)lire les articles suivants avant d'aborder le travail :  [[Ronneberger2015]](https://arxiv.org/abs/1505.04597), [[Dosovitskiy2020]](https://arxiv.org/abs/2010.11929) et, pour l'encodage multimodal, [[Jaegle2021]](https://arxiv.org/abs/2103.03206).
  Ces lectures permettront de mieux comprendre les codes, de se donner des pistes d'amélioration et de pouvoir répondre aux questions lors de l'oral.

## Annexe A : code d'un UNet

In [ ]:
################################   UNet (parties)###############################
import torch
import torch.nn as nn
import torch.nn.functional as F

class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x

class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x

class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x



class Up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(Up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear')
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, kernel_size=2, stride=2)

        self.conv = double_conv(2*in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffX = x1.size()[2] - x2.size()[2]
        diffY = x1.size()[3] - x2.size()[3]
        x2 = F.pad(x2, (diffX // 2, int(diffX / 2),
                        diffY // 2, int(diffY / 2)))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)


    def forward(self, x):
        x = self.conv(x)
        return x


################################################################################
########################################   Mini Unet  ##########################

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes,size=64):
        super(UNet, self).__init__()
        self.inc = inconv(n_channels, size)
        self.down1 = Down(size, 2*size)
        self.down2 = Down(2*size, 4*size)
        self.down3 = Down(4*size, 8*size)
        self.down4 = Down(8*size, 8*size)
        self.up1 = Up(8*size, 4*size)
        self.up2 = Up(4*size, 2*size)
        self.up3 = Up(2*size, size)
        self.up4 = Up(size, size)
        self.outc = outconv(size, n_classes)
        self.outc2 = outconv(size, n_classes)
        self.n_classes=n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return   x

In [ ]:
# Exemple d'instanciation :
ch_in = 1
ch_out = 1
size = 16

fcn = UNet(ch_in, ch_out, size)

## Annexe B : exemple d'un visual transformer adapté au problème $\mathcal{P}_3$

In [ ]:
# Paramètres du modèle :
image_size = [64,64]
channels = 1
patch_size = 4
d_model = 120
mlp_expansion_ratio = 4
d_ff = mlp_expansion_ratio * d_model
n_heads = 4
n_layers = 12

In [ ]:
# Module interne du réseau responsable de l'encodage des variables :
from exam_S3.utile_Transformers import UnifiedEmbedding
ue = UnifiedEmbedding(d_model, patch_size, channels)
_, xp, xs, _, xi = gen_image_with_pairs(6, n_pairs, n_points)

embeddings = ue(xs, xp, xi)
print(embeddings.shape)


In [ ]:
from exam_S3.utile_Transformers import FusionTransformer
model = FusionTransformer(image_size, patch_size, n_layers, d_model, d_ff, n_heads, channels=1)
_, xp, xs, _, xi = gen_image_with_pairs(6, n_pairs, n_points)
print(model(xs, xp, xi).shape)


## Annexe C : téléchargement des poids d'un visual transformer pré-entraîné.

In [ ]:
# FusionTransformers sur 900 époques :
# mViT_900ep.pth : entraîné sur P3
! wget https://www.grosfichiers.com/K3aaxZcSnX4_J2FzgRR6Mkk
! unzip K3aaxZcSnX4_J2FzgRR6Mkk
! rm K3aaxZcSnX4_J2FzgRR6Mkk

In [ ]:
! ls